In [17]:
import sys
sys.path.insert(1, '../src/')

import db_wrapper as db
import gemini_wrapper as gemini
import re

In [ ]:
def compose_info(db_answer):
    info = ''
    for _, doc, page, txt, _ in db_answer:
        info += '\n'
        info += f'## Document {doc}, Page {page+1}\n'
        info += txt
        info += '\n'

    return info

def compose_prompt(query, db_answer):
    template = open('../metadata/template.md').read()
    ans_template = open('../metadata/answer_template.json').read()
    info = compose_info(db_answer)
    prompt = template.format(query=query, ans_template=ans_template, info=info)

    return prompt

def extract_json(txt):
    m = re.match('```json(.+)```', txt, re.DOTALL)
    if m is None:
        return txt
    
    return m.group(1).strip()

In [4]:
gemini.configure()

In [5]:
query = 'What is the maximum rating for RCBO in the small power system?'
emb = gemini.create_embedding(query)

In [6]:
r = db.semantic_search(emb)

In [9]:
info = compose_info(r)
open('../tmp/info.md', 'w+').write(info)

28250

In [14]:
prompt = compose_prompt(query, r)
open('../tmp/prompt.md', 'w+').write(prompt)

28634

In [ ]:
answer = gemini.ask(prompt)
print(answer)
open('../tmp/answer.json', 'w+').write(extract_json(answer))

```json
{
  "page": 25,
  "doc": 0,
  "answer": "The rating of RCBOs does not exceed 30mA for residual currents."
}
```



120